# 모델 파라미터 측정

In [12]:
import torch
import json
import sys

from os import path
sys.path.append(r"D:\박준호\2024년\NLP-Standard-Dialect-Transformation")

import translate_model
import vectorizer

In [14]:
vectorizer_filepath = "../model_storage/dia_to_sta/jeju/jamo/vectorizer.json"
with open(vectorizer_filepath, encoding="utf-8") as fp:
    vectorizer_val = vectorizer.NMTVectorizer.from_serializable(json.load(fp))
    
train_result_path = "../model_storage/dia_to_sta/jeju/jamo/logs/train_at_2025-01-06_08_54.json"
with open(train_result_path, mode="r+", encoding="utf-8") as fp:
        train_result_dict = json.loads(fp.read())

model = translate_model.NMTModel(source_vocab_size=len(vectorizer_val.source_vocab), 
                 source_embedding_size=train_result_dict["source_embedding_size"], 
                 target_vocab_size=len(vectorizer_val.target_vocab),
                 target_embedding_size=train_result_dict["target_embedding_size"], 
                 encoding_size=train_result_dict["encoding_size"],
                 target_bos_index=vectorizer_val.target_vocab.begin_seq_index,
                 target_eos_index=vectorizer_val.target_vocab.end_seq_index,
                 max_gen_length=vectorizer_val.max_target_length + 1)

In [28]:
from torchsummaryX import summary

summary(model, torch.zeros((2, 907), dtype=torch.int64), x_source_lengths=torch.Tensor([81, 83]).to(torch.int64))

c:\anaconda3\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


                                                Kernel Shape   Output Shape  \
Layer                                                                         
0_encoder.Embedding_source_embedding               [64, 156]   [2, 907, 64]   
1_encoder.Linear_embed_to_rnn                      [64, 128]  [2, 907, 128]   
2_encoder.multi_rnn.0.Linear_to_hidden_and_gate   [128, 512]  [2, 907, 512]   
3_encoder.multi_rnn.0.Linear_to_out               [256, 128]  [2, 907, 128]   
4_encoder.multi_rnn.1.Linear_to_hidden_and_gate   [128, 512]  [2, 907, 512]   
...                                                      ...            ...   
2743_decoder.GRUCell_gru_cell                              -       [2, 256]   
2744_decoder.Linear_classifier                    [512, 156]       [2, 156]   
2745_decoder.Embedding_target_embedding            [64, 156]        [2, 64]   
2746_decoder.GRUCell_gru_cell                              -       [2, 256]   
2747_decoder.Linear_classifier                    [5

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder.Embedding_source_embedding,"[64, 156]","[2, 907, 64]",9984.0,9984
1_encoder.Linear_embed_to_rnn,"[64, 128]","[2, 907, 128]",8320.0,8192
2_encoder.multi_rnn.0.Linear_to_hidden_and_gate,"[128, 512]","[2, 907, 512]",65536.0,65536
3_encoder.multi_rnn.0.Linear_to_out,"[256, 128]","[2, 907, 128]",32768.0,32768
4_encoder.multi_rnn.1.Linear_to_hidden_and_gate,"[128, 512]","[2, 907, 512]",65536.0,65536
...,...,...,...,...
2743_decoder.GRUCell_gru_cell,-,"[2, 256]",NaN,442368
2744_decoder.Linear_classifier,"[512, 156]","[2, 156]",NaN,79872
2745_decoder.Embedding_target_embedding,"[64, 156]","[2, 64]",NaN,9984


In [29]:
print(len(vectorizer_val.source_vocab))
print(len(vectorizer_val.target_vocab))

156
156


## sentencepiece 모델 

In [30]:
vectorizer_filepath = "../model_storage/dia_to_sta/jeju/SentencePiece/vectorizer.json"
with open(vectorizer_filepath, encoding="utf-8") as fp:
    vectorizer_val = vectorizer.NMTVectorizer.from_serializable(json.load(fp))
    
train_result_path = "../model_storage/dia_to_sta/jeju/SentencePiece/logs/train_at_2025-01-04_07_28.json"
with open(train_result_path, mode="r+", encoding="utf-8") as fp:
        train_result_dict = json.loads(fp.read())

model = translate_model.NMTModel(source_vocab_size=len(vectorizer_val.source_vocab), 
                 source_embedding_size=train_result_dict["source_embedding_size"], 
                 target_vocab_size=len(vectorizer_val.target_vocab),
                 target_embedding_size=train_result_dict["target_embedding_size"], 
                 encoding_size=train_result_dict["encoding_size"],
                 target_bos_index=vectorizer_val.target_vocab.begin_seq_index,
                 target_eos_index=vectorizer_val.target_vocab.end_seq_index,
                 max_gen_length=vectorizer_val.max_target_length + 1)

In [36]:
sp_df = summary(model, torch.zeros((2, 907), dtype=torch.int64), x_source_lengths=torch.Tensor([81, 83]).to(torch.int64))

                                                    Kernel Shape  \
Layer                                                              
0_encoder.Embedding_source_embedding                 [64, 31486]   
1_encoder.Linear_embed_to_rnn                          [64, 128]   
2_encoder.multi_rnn.0.Linear_to_hidden_and_gate       [128, 512]   
3_encoder.multi_rnn.0.Linear_to_out                   [256, 128]   
4_encoder.multi_rnn.1.Linear_to_hidden_and_gate       [128, 512]   
5_encoder.multi_rnn.1.Linear_to_out                   [256, 128]   
6_encoder.multi_rnn.2.Linear_to_hidden_and_gate       [128, 512]   
7_encoder.multi_rnn.2.Linear_to_out                   [256, 128]   
8_encoder.reverse_rnn.0.Linear_to_hidden_and_gate     [128, 512]   
9_encoder.reverse_rnn.0.Linear_to_out                 [256, 128]   
10_encoder.reverse_rnn.1.Linear_to_hidden_and_gate    [128, 512]   
11_encoder.reverse_rnn.1.Linear_to_out                [256, 128]   
12_encoder.reverse_rnn.2.Linear_to_hidden_and_ga

c:\anaconda3\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


In [35]:
sp_df

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder.Embedding_source_embedding,"[64, 31486]","[2, 907, 64]",2015104.0,2015104
1_encoder.Linear_embed_to_rnn,"[64, 128]","[2, 907, 128]",8320.0,8192
2_encoder.multi_rnn.0.Linear_to_hidden_and_gate,"[128, 512]","[2, 907, 512]",65536.0,65536
3_encoder.multi_rnn.0.Linear_to_out,"[256, 128]","[2, 907, 128]",32768.0,32768
4_encoder.multi_rnn.1.Linear_to_hidden_and_gate,"[128, 512]","[2, 907, 512]",65536.0,65536
...,...,...,...,...
397_decoder.GRUCell_gru_cell,-,"[2, 256]",NaN,442368
398_decoder.Linear_classifier,"[512, 31049]","[2, 31049]",NaN,15897088
399_decoder.Embedding_target_embedding,"[64, 31049]","[2, 64]",NaN,1987136
